# Package List

In [1]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

/home/jellybee/anaconda3/envs/llm_311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [2]:
df = pd.read_csv("../data/서울시대여공구찾기정보.csv", encoding='cp949')

## Preprocessing

In [3]:
del_col = df.isnull().sum()[df.isnull().sum()/df.shape[0]>0.95].index.tolist()
df = df[[x for x in df.columns if x not in del_col]]
df['생성일시'] = pd.to_datetime(df['생성일시'], format='%Y%m%d%H%M%S')
del df['위치(위도,경도)']
del df['요금']

In [4]:
df['요금(과금기준 + 요금)'].fillna('0원', inplace=True)
def split_period_and_price(x):
    if '1일' in x:
        return pd.Series(['1일', x.replace('1일', '').strip()])
    elif '1시간' in x:
        return pd.Series(['1시간', x.replace('1시간', '').strip()])
    else:
        return pd.Series(['없음', x.strip()])

df[['과금기준', '요금']] = df['요금(과금기준 + 요금)'].apply(split_period_and_price)
del df['요금(과금기준 + 요금)']

# Location Info 

In [5]:
import requests

def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '강남역'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


강남역 위도/경도: (37.4999282, 127.0263182)


In [26]:
api_key =''
openai.api_key = api_key

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI(api_key=api_key)

In [23]:
def invoke(question, model="gpt-4o", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages = [
                    {"role": "system", 
                        "content": "너는 사람들의 질문에 친절히 답해주는 도우미야."},
                    {"role": "user", 
                    "content": question}
                ],
        temperature=temperature
    )
    return response.choices[0].message.content

In [ ]:
question = "주식에 대해서 공부하고싶어. 회사의 재무재표를 보는 방법에 대해서 상세히 설명해줄래?"

In [25]:
answer = invoke(question)
print("💬 GPT 응답:", answer)

💬 GPT 응답: 재무제표는 회사의 재무 상태와 경영 성과를 파악할 수 있는 중요한 도구입니다. 주요 재무제표에는 대차대조표, 손익계산서, 현금흐름표가 포함됩니다. 각각의 재무제표를 어떻게 분석하는지 설명해드릴게요.

### 1. 대차대조표 (Balance Sheet)
대차대조표는 특정 시점에서 회사의 자산, 부채, 자본을 보여줍니다. 

- **자산 (Assets):** 회사가 소유하고 있는 모든 자원입니다. 유동자산(현금, 매출채권 등)과 비유동자산(건물, 장비 등)으로 구분됩니다.
- **부채 (Liabilities):** 회사가 갚아야 할 채무입니다. 유동부채(단기 차입금, 미지급금 등)와 비유동부채(장기 차입금 등)로 구분됩니다.
- **자본 (Equity):** 자산에서 부채를 뺀 것으로, 주주들이 회사에 투자한 자본과 이익잉여금으로 구성됩니다.

**분석 포인트:**
- **유동비율:** 유동자산/유동부채. 1 이상의 값은 단기 채무를 갚을 수 있는 능력을 의미합니다.
- **부채비율:** 부채/자본. 낮을수록 재무 안정성이 높습니다.

### 2. 손익계산서 (Income Statement)
손익계산서는 일정 기간 동안의 수익, 비용, 이익을 보여줍니다.

- **매출 (Revenue):** 제품이나 서비스를 판매하여 얻은 총 수익입니다.
- **매출원가 (Cost of Goods Sold, COGS):** 제품을 생산하거나 서비스를 제공하는 데 들어간 직접 비용입니다.
- **영업이익 (Operating Income):** 매출에서 매출원가와 영업비용을 뺀 금액으로, 회사의 기본 영업활동에서 발생한 이익입니다.
- **순이익 (Net Income):** 모든 수익에서 모든 비용과 세금을 뺀 최종 이익입니다.

**분석 포인트:**
- **영업이익률:** 영업이익/매출. 영업 활동의 효율성을 나타냅니다.
- **순이익률:** 순이익/매출. 전체적인 수익성을 보여줍니다.

### 3. 현금흐름표 (Cash Flow Statement)
현금흐름표는 

In [13]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response

In [14]:
output = invoke('배가 고픈데 생선을 먹고싶어. 6월에 먹을 수 있는 제철 생선 있어?')

TypeError: Responses.create() got an unexpected keyword argument 'messages'

In [20]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야돼
    """
    response = openai.ChatCompletion.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "당신은 완벽한 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    result_text = response['choices'][0]['message']['content']

    try:
        result_dict = json.loads(result_text)
        with open("clustered_equipment.json", "w", encoding="utf-8") as f:
            json.dump(result_dict, f, ensure_ascii=False, indent=2)
        print("✅ 클러스터링 결과 저장 완료: clustered_equipment.json")
        return result_dict
    except json.JSONDecodeError:
        print("⚠️ JSON 형식 오류. 출력 확인 필요:")
        print(result_text)
        return None
    

In [19]:
question = '나 너무 배고파'
result = invoke(question)

# 만약 JSON 결과를 바로 보고 싶으면
import pandas as pd

# if result:
#     df_result = pd.DataFrame(result)
#     display(df_result)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [25]:
prompt = f"""
    너는 도구가 어디있는지 찾아주는 도우미야. 
    사용자가 필요한 도구와 현재 위치를 알려주거야.
    너는 {df}를 참고해서 **사용자가 필요로 하는 도구**와 **현재 위치**에서 가장 가까이에있는 대여소를 찾아주면 돼.

    """